In [27]:
# Import necessary libraries and modules
import json
import pickle
import numpy as np
from tensorflow.keras.models import load_model
from nltk.stem import WordNetLemmatizer
import random
import nltk
nltk.download('punkt')

# Define paths for the files
model_path = '../chatbot/chatbot_model.h5'
words_path = '../chatbot/words.pkl'
classes_path = '../chatbot/classes.pkl'
intents_path = '../chatbot/intents.json'

# Load the trained model and the necessary data
model = load_model(model_path)
words = pickle.load(open(words_path, 'rb'))
classes = pickle.load(open(classes_path, 'rb'))
intents = json.loads(open(intents_path).read())
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jbaru\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
# Define a function for tokenization and lemmatization
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [29]:
# Define a function to create a bag of words
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s: 
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {word}")
    return(np.array(bag))


In [30]:
# Define a function for predicting the class
def predict_class(sentence, model, words, classes):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [31]:
# Define a function to get the response from the chatbot
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [34]:
# Testing the features
test_sentences = ["Hi there", "What are your hours?", "What is AI?"] # Users can add were sentences to test
for sentence in test_sentences:
    print(f"Sentence: {sentence}")
    # Get the class
    ints = predict_class(sentence, model, words, classes)
    # Get the response
    res = get_response(ints, intents)
    print(f"Response: {res}")

Sentence: Hi there
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Response: Hi! I am CentBot, How can I help you?
Sentence: What are your hours?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Response: Hi! I am CentBot, How can I help you?
Sentence: What is AI?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Response: Hi! I am CentBot, How can I help you?
